# Study force-controlled single facet

In [1]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
import warnings
import matplotlib.pylab as plt

In [2]:
from ibvpy.api import TStepBC, TFCyclicNonsymmetricConstant, TFBilinear
from ibvpy.bcond import BCSlice, BCDof
from ibvpy.xmodel.xdomain_fe_grid import XDomainFEGrid
from ibvpy.xmodel.xdomain_interface import XDomainFEInterface

In [3]:
from ibvpy.fets import FETS2D4Q
from ibvpy.fets import FETS3D8H
from ibvpy.fets.fets1D5 import FETS1D52ULRH

In [4]:
from bmcs_matmod.slide.vslide_34 import Slide34
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic
from ibvpy.tmodel.mats2D.mats2D_elastic.vmats2D_elastic import \
    MATS2DElastic
from bmcs_utils.api import Model
from bmcs_matmod.slide.slide_explorer import SlideExplorer

In [5]:
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField

# Monotonic loading

In [6]:
n_e = 1
L_x = 1.0

xd_lower = XDomainFEGrid(coord_min=(0, -1),
                          coord_max=(1, 0),
                          shape=(1, 1),
                          fets=FETS2D4Q())
xd_upper = XDomainFEGrid(coord_min=(0, 0),
                          coord_max=(1, 1),
                          shape=(1, 1),
                          fets=FETS2D4Q())

m_1 = MATS2DElastic(E=40000, nu=0.2)
m_1 = MATS2DElastic(E=40000, nu=0.2)

xd12 = XDomainFEInterface(
    I=xd_lower.mesh.I[-1, :],
    J=xd_upper.mesh.I[0, :],
    fets=FETS1D52ULRH()
)

material_params = dict(
     E_T=100, gamma_T=200, K_T=0, S_T=0.05, c_T=3, bartau=3, 
     E_N=100, S_N=0.05, c_N = 3, m = 0.15, f_t=5, f_c=80, f_c0 = 40, eta=0.2)


bond_m = Slide34(**material_params)

m = TStepBC(
    domains=[
        (xd_lower, m_1),
        (xd_upper, m_1),
        (xd12, bond_m),
    ],
)


load_dim = 0 # 
fixed_dim = 1 # 0 - x, 1 - y

upper_control_slice = BCSlice(slice=xd_upper.mesh[:, :, :, :],
                          var='u', dims=[load_dim], value=0)
upper_control_dofs = upper_control_slice.dofs
upper_control = [BCDof(var='u', dof=dof, value = -0.4) 
                 for dof in upper_control_dofs ]

upper_compression_slice = BCSlice(slice=xd_upper.mesh[:, :, :, :],
         var='u', dims=[fixed_dim], value=0)
upper_compression_dofs = upper_compression_slice.dofs
upper_compression = [BCDof(var='u', dof=dof, value = 0.0) 
                 for dof in upper_compression_dofs ]

lower_fixed = BCSlice(slice=xd_lower.mesh[:, :, :, :],
         var='u', dims=[0,1], value=0)

bc1 = upper_control + upper_compression + [lower_fixed]

m.bc=bc1
m.hist.vis_record = {
    'strain': Vis3DTensorField(var='eps_ab'),
}
s = m.sim
s.tloop.verbose = False # set false to avoid the prints during each increment
s.tloop.k_max = 50
s.tline.step = 0.01
s.reset()
s.run()


In [7]:
se1 = SlideExplorer(n_steps=100, k_max=50)
se1.slide_model.trait_set(**material_params)
se1.trait_set(s_x_1 = 0.4, s_y_1 = 0, w_1 = 0);
se1.run()

In [9]:
se1.interact()

In [8]:
import matplotlib.pylab as plt

tau_x_pi_1, tau_y_pi_1, sig_pi_1, Z_1, X_x_1, X_y_1, Y_T_1, Y_N_1 = se1.Sig_arr.T
s_x_pi_, s_y_pi_, w_pi_, z_, alpha_x_, alpha_y_, omega_T_, omega_N_ = se1.Eps_arr.T

n_step_1 = len(tau_x_pi_1)

w_1 = se1.w_t
s_x_1 = se1.s_x_t.T
   
F_to = m.hist.F_t
U_to = m.hist.U_t
F_l_t = np.sum(F_to[:, upper_control_dofs], axis=-1)
U_r_t = np.average(U_to[:, upper_control_dofs], axis=-1)

states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}

time = m.hist.t

fig, ((ax1,  ax2, ax3,  ax4)) =  plt.subplots(1,4, figsize=(14, 4))
fig.canvas.header_visible=False

ax1.plot(U_r_t, EpsSig_t['tau_pi_x'][:,0,0], label='FE')
ax1.plot(U_r_t, tau_x_pi_1, label='Material Point')
ax1.set_xlabel('u_x')
ax1.set_ylabel('$\sigma_T$')
ax1.legend()

ax2.plot(U_r_t, EpsSig_t['sig_pi'][:,0,0])
ax2.plot(U_r_t, sig_pi_1)
ax2.set_xlabel('u_x')
ax2.set_ylabel('$\sigma_N$')

ax3.plot(U_r_t, EpsSig_t['omega_T'][:,0,0])
ax3.plot(U_r_t, omega_T_)
ax3.set_xlabel('s_x')
ax3.set_ylabel('$\omega_T$')


ax4.plot(U_r_t, EpsSig_t['omega_N'][:,0,0])
ax4.plot(U_r_t, omega_N_)
ax4.set_xlabel('s_x')
ax4.set_ylabel('$\omega_N$')

fig.tight_layout()


#fname= 'ElsevierFont.pdf'
#plt.savefig(fname, bbox_inches='tight', pad_inches=0.01/2.54, dpi=300)
##get the link to the PDF
#from IPython.display import FileLink
#FileLink(fname)
#p#lt.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
tf_first = TFBilinear(loading_ratio=1.0, time_ratio=0.5)
tf_second = TFBilinear(loading_ratio=0.00, time_ratio=0.5)

# Pre-crompression (DC) and then monotonic loading (DC)

In [28]:
n_e = 1
L_x = 1.0

xd_lower = XDomainFEGrid(coord_min=(0, -1),
                          coord_max=(1, 0),
                          shape=(1, 1),
                          fets=FETS2D4Q())
xd_upper = XDomainFEGrid(coord_min=(0, 0),
                          coord_max=(1, 1),
                          shape=(1, 1),
                          fets=FETS2D4Q())

m_1 = MATS2DElastic(E=40000, nu=0.2)
m_2 = MATS2DElastic(E=40000, nu=0.2)

xd12 = XDomainFEInterface(
    I=xd_lower.mesh.I[-1, :],
    J=xd_upper.mesh.I[0, :],
    fets=FETS1D52ULRH()
)

material_params = dict(
     E_T=100, gamma_T=200, K_T=0, S_T=0.05, c_T=3, bartau=3, 
     E_N=100, S_N=0.05, c_N = 3, m = 0.15, f_t=5, f_c=80, f_c0 = 40, eta=0.2)


bond_m = Slide34(**material_params)

m1 = TStepBC(
    domains=[
        (xd_lower, m_1),
        (xd_upper, m_1),
        (xd12, bond_m),
    ],
)
    
load_dim = 0 # 
fixed_dim = 1 # 0 - x, 1 - y

upper_control_slice = BCSlice(slice=xd_upper.mesh[:, :, :, :],
                          var='u', dims=[load_dim], value=0)
upper_control_dofs = upper_control_slice.dofs
upper_control = [BCDof(var='u', dof=dof, value = -0.4, time_function=tf_second) 
                 for dof in upper_control_dofs ]

F = -40 / len(upper_control_dofs)

upper_compression_slice = BCSlice(slice=xd_upper.mesh[:, :, :, :],
         var='u', dims=[fixed_dim], value=0)
upper_compression_dofs = upper_compression_slice.dofs
max_w = -material_params['f_c'] / material_params['E_N'] * 0.5
upper_compression = [BCDof(var='u', dof=dof, value = max_w, time_function=tf_first) 
                 for dof in upper_compression_dofs ]

lower_fixed = BCSlice(slice=xd_lower.mesh[:, :, :, :],
         var='u', dims=[0,1], value=0)

bc1 = upper_control + upper_compression + [lower_fixed]

m1.bc=bc1
m1.hist.vis_record = {
    'strain': Vis3DTensorField(var='eps_ab'),
}
s1 = m1.sim
s1.tloop.verbose = True # set false to avoid the prints during each increment
s1.tloop.k_max = 50
s1.tline.step = 0.01
s1.reset()
s1.run()

t:	 0.00(0), 
	 0.01(1), 
	 0.02(1), 
	 0.03(1), 
	 0.04(1), 
	 0.05(1), 
	 0.06(1), 
	 0.07(1), 
	 0.08(1), 
	 0.09(1), 
	 0.10(1), 
	 0.11(1), 
	 0.12(1), 
	 0.13(1), 
	 0.14(1), 
	 0.15(1), 
	 0.16(1), 
	 0.17(1), 
	 0.18(1), 
	 0.19(1), 
	 0.20(1), 
	 0.21(1), 
	 0.22(1), 
	 0.23(1), 
	 0.24(1), 
	 0.25(1), 
	 0.26(1), 
	 0.27(1), 
	 0.28(1), 
	 0.29(1), 
	 0.30(1), 
	 0.31(1), 
	 0.32(1), 
	 0.33(1), 
	 0.34(1), 
	 0.35(1), 
	 0.36(1), 
	 0.37(1), 
	 0.38(1), 
	 0.39(1), 
	 0.40(1), 
	 0.41(1), 
	 0.42(1), 
	 0.43(1), 
	 0.44(1), 
	 0.45(1), 
	 0.46(1), 
	 0.47(1), 
	 0.48(1), 
	 0.49(1), 
	 0.50(1), 
	 0.51(1), 
	 0.52(1), 
	 0.53(1), 
	 0.54(1), 
	 0.55(1), 
	 0.56(1), 
	 0.57(1), 
	 0.58(1), 
	 0.59(1), 
	 0.60(1), 
	 0.61(1), 
	 0.62(1), 
	 0.63(1), 
	 0.64(1), 
	 0.65(1), 
	 0.66(1), 
	 0.67(1), 
	 0.68(1), 
	 0.69(1), 
	 0.70(1), 
	 0.71(1), 
	 0.72(1), 
	 0.73(1), 
	 0.74(1), 
	 0.75(1), 
	 0.76(1), 
	 0.77(1), 
	 0.78(1), 
	 0.79(1), 
	 0.80(1), 
	 0.81(1), 
	 0.82(1), 
	 

# Pre-crompression (FC) and then monotonic loading (DC)

In [10]:
n_e = 1
L_x = 1.0

xd_lower = XDomainFEGrid(coord_min=(0, -1),
                          coord_max=(1, 0),
                          shape=(1, 1),
                          fets=FETS2D4Q())
xd_upper = XDomainFEGrid(coord_min=(0, 0),
                          coord_max=(1, 1),
                          shape=(1, 1),
                          fets=FETS2D4Q())

m_1 = MATS2DElastic(E=40000, nu=0.2)
m_2 = MATS2DElastic(E=40000, nu=0.2)

xd12 = XDomainFEInterface(
    I=xd_lower.mesh.I[-1, :],
    J=xd_upper.mesh.I[0, :],
    fets=FETS1D52ULRH()
)

material_params = dict(
     E_T=100, gamma_T=200, K_T=0, S_T=0.05, c_T=3, bartau=3, 
     E_N=100, S_N=0.05, c_N = 3, m = 0.15, f_t=5, f_c=80, f_c0 = 40, eta=0.2)


bond_m = Slide34(**material_params)

m2 = TStepBC(
    domains=[
        (xd_lower, m_1),
        (xd_upper, m_1),
        (xd12, bond_m),
    ],
)

tf_force = TFBilinear(loading_ratio=1.0, time_ratio=0.5)
tf_disp = TFBilinear(loading_ratio=0.00, time_ratio=0.5)
    
load_dim = 0 # 
fixed_dim = 1 # 0 - x, 1 - y

upper_control_slice = BCSlice(slice=xd_upper.mesh[:, :, :, :],
                          var='u', dims=[load_dim], value=0)
upper_control_dofs = upper_control_slice.dofs
upper_control = [BCDof(var='u', dof=dof, value = -0.4, time_function=tf_disp) 
                 for dof in upper_control_dofs ]

F = -40 / len(upper_control_dofs)

upper_compression_slice = BCSlice(slice=xd_upper.mesh[:, :, :, :],
         var='u', dims=[fixed_dim], value=0)
upper_compression_dofs = upper_compression_slice.dofs
upper_compression = [BCDof(var='f', dof=dof, value = F, time_function=tf_force) 
                 for dof in upper_compression_dofs ]

lower_fixed = BCSlice(slice=xd_lower.mesh[:, :, :, :],
         var='u', dims=[0,1], value=0)

bc1 = upper_control + upper_compression + [lower_fixed]

m2.bc=bc1
m2.hist.vis_record = {
    'strain': Vis3DTensorField(var='eps_ab'),
}
s2 = m2.sim
s2.tloop.verbose = True # set false to avoid the prints during each increment
s2.tloop.k_max = 50
s2.tline.step = 0.01
s2.reset()
s2.run()

t:	 0.00(0), 
	 0.01(1), 
	 0.02(1), 
	 0.03(1), 
	 0.04(1), 
	 0.05(1), 
	 0.06(1), 
	 0.07(1), 
	 0.08(1), 
	 0.09(1), 
	 0.10(1), 
	 0.11(1), 
	 0.12(1), 
	 0.13(1), 
	 0.14(1), 
	 0.15(1), 
	 0.16(1), 
	 0.17(1), 
	 0.18(1), 
	 0.19(1), 
	 0.20(1), 
	 0.21(1), 
	 0.22(1), 
	 0.23(1), 
	 0.24(1), 
	 0.25(1), 
	 0.26(1), 
	 0.27(1), 
	 0.28(1), 
	 0.29(1), 
	 0.30(1), 
	 0.31(1), 
	 0.32(1), 
	 0.33(1), 
	 0.34(1), 
	 0.35(1), 
	 0.36(1), 
	 0.37(1), 
	 0.38(1), 
	 0.39(1), 
	 0.40(1), 
	 0.41(1), 
	 0.42(1), 
	 0.43(1), 
	 0.44(1), 
	 0.45(1), 
	 0.46(1), 
	 0.47(1), 
	 0.48(1), 
	 0.49(1), 
	 0.50(1), 
	 0.51(1), 
	 0.52(1), 
	 0.53(1), 
	 0.54(1), 
	 0.55(1), 
	 0.56(1), 
	 0.57(1), 
	 0.58(1), 
	 0.59(1), 
	 0.60(1), 
	 0.61(1), 
	 0.62(3), 
	 0.63(2), 
	 0.64(3), 
	 0.65(3), 
	 0.66(3), 
	 0.67(3), 
	 0.68(3), 
	 0.69(3), 
	 0.70(3), 
	 0.71(3), 
	 0.72(3), 
	 0.73(3), 
	 0.74(4), 
	 0.75(4), 
	 0.76(4), 
	 0.77(4), 
	 0.78(4), 
	 0.79(4), 
	 0.80(4), 
	 0.81(4), 
	 0.82(4), 
	 

In [11]:
n_e = 1
L_x = 1.0

xd_lower = XDomainFEGrid(coord_min=(0, -1),
                          coord_max=(1, 0),
                          shape=(1, 1),
                          fets=FETS2D4Q())
xd_upper = XDomainFEGrid(coord_min=(0, 0),
                          coord_max=(1, 1),
                          shape=(1, 1),
                          fets=FETS2D4Q())

m_1 = MATS2DElastic(E=40000, nu=0.2)
m_2 = MATS2DElastic(E=40000, nu=0.2)

xd12 = XDomainFEInterface(
    I=xd_lower.mesh.I[-1, :],
    J=xd_upper.mesh.I[0, :],
    fets=FETS1D52ULRH()
)

material_params = dict(
     E_T=100, gamma_T=200, K_T=0, S_T=0.05, c_T=3, bartau=3, 
     E_N=100, S_N=0.05, c_N = 3, m = 0.15, f_t=5, f_c=80, f_c0 = 40, eta=0.2)


bond_m = Slide34(**material_params)

m3 = TStepBC(
    domains=[
        (xd_lower, m_1),
        (xd_upper, m_1),
        (xd12, bond_m),
    ],
)

tf_force = TFBilinear(loading_ratio=1.0, time_ratio=0.5)
tf_disp = TFBilinear(loading_ratio=0.00, time_ratio=0.5)
    
load_dim = 0 # 
fixed_dim = 1 # 0 - x, 1 - y

upper_control_slice = BCSlice(slice=xd_upper.mesh[:, :, :, :],
                          var='u', dims=[load_dim], value=0)
upper_control_dofs = upper_control_slice.dofs
tau = -10 / len(upper_control_dofs)
upper_control = [BCDof(var='f', dof=dof, value = tau, time_function=tf_disp) 
                 for dof in upper_control_dofs ]

F = -40 / len(upper_control_dofs)

upper_compression_slice = BCSlice(slice=xd_upper.mesh[:, :, :, :],
         var='u', dims=[fixed_dim], value=0)
upper_compression_dofs = upper_compression_slice.dofs
upper_compression = [BCDof(var='f', dof=dof, value = F, time_function=tf_force) 
                 for dof in upper_compression_dofs ]

lower_fixed = BCSlice(slice=xd_lower.mesh[:, :, :, :],
         var='u', dims=[0,1], value=0)

bc1 =  upper_compression + [lower_fixed]

m3.bc=bc1
m3.hist.vis_record = {
    'strain': Vis3DTensorField(var='eps_ab'),
}
s3 = m3.sim
s3.tloop.verbose = True # set false to avoid the prints during each increment
s3.tloop.k_max = 50
s3.tline.step = 0.01
s3.reset()
s3.run()


t:	 0.00(0), 
	 0.01(1), 
	 0.02(1), 
	 0.03(1), 
	 0.04(1), 
	 0.05(8), 
	 0.06(10), 
	 0.07(11), 
	 0.08(12), 
	 0.09(15), 
	 0.10(23), 
	 0.11

RuntimeError: Factor is exactly singular

In [12]:
se2 = SlideExplorer(n_steps=50, k_max=50)
se2.slide_model.trait_set(**material_params)
se2.trait_set(s_x_1 = 0.0, s_y_1 = 0, w_1 = -material_params['f_c'] / material_params['E_N'] * 0.5);
se2.run()
se2.trait_set(s_x_1 = 0.4, s_y_1 = 0, w_1 = -material_params['f_c'] / material_params['E_N'] * 0.5);
se2.run()

In [20]:
se2.interact()

In [34]:
import matplotlib.pylab as plt

tau_x_pi_2, tau_y_pi_2, sig_pi_2, Z_2, X_x_2, X_y_2, Y_T_2, Y_N_2 = se2.Sig_arr.T
s_x_pi_2, s_y_pi_2, w_pi_2, z_2, alpha_x_2, alpha_y_2, omega_T_2, omega_N_2 = se2.Eps_arr.T

w_2 = se2.w_t
s_x_2 = se2.s_x_t.T

mx = m3
F_to_2 = mx.hist.F_t
U_to_2 = mx.hist.U_t
F_l_t_2 = np.sum(F_to_2[:, upper_control_dofs], axis=-1)
s_FE_t = np.average(U_to_2[:, upper_control_dofs], axis=-1)
w_FE_t = np.average(U_to_2[:, upper_compression_dofs], axis=-1)

states_t_2 = [states_t[2] for states_t in mx.hist.state_vars]
var_names_2 = states_t_2[0].keys()
EpsSig_t_2 = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t_2 ]) 
    for var_name in var_names_2
}

time = mx.hist.t

fig, ((ax1,  ax2, ax3,  ax4)) =  plt.subplots(1,4, figsize=(14, 4))
fig.canvas.header_visible=False

ax1.plot(-s_FE_t, EpsSig_t_2['tau_pi_x'][:,0,0], label='FE')
ax1.plot(s_x_2, tau_x_pi_2, label='Material Point')
ax1.set_xlabel('s_x')
ax1.set_ylabel('$\sigma_T$')
ax1.legend()

ax2.plot(w_FE_t, EpsSig_t_2['sig_pi'][:,0,0])
ax2.plot(w_2, sig_pi_2)
ax2.set_xlabel('w')
ax2.set_ylabel('$\sigma_N$')

ax3.plot(s_FE_t, EpsSig_t_2['omega_T'][:,0,0])
ax3.plot(-s_x_2, omega_T_2)
ax3.set_xlabel('s_x')
ax3.set_ylabel('$\omega_T$')

ax4.plot(s_FE_t, EpsSig_t_2['omega_N'][:,0,0])
ax4.plot(-s_x_2, omega_N_2)
ax4.set_xlabel('s_x')
ax4.set_ylabel('$\omega_N$')

fig.tight_layout()


#fname= 'ElsevierFont.pdf'
#plt.savefig(fname, bbox_inches='tight', pad_inches=0.01/2.54, dpi=300)
##get the link to the PDF
#from IPython.display import FileLink
#FileLink(fname)
#p#lt.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
tf_force.interact()